# 02 - Label-Preserving Synthetic Extreme Weather Generation

## Pipeline 1: KITTI Adverse Weather Data Augmentation

**Goal**: Generate photorealistic extreme-weather versions of KITTI images while **preserving original ground-truth labels** (bounding boxes / classes).

### Pipeline Steps:
1. **Object Segmentation (SAM ViT-H)**: Precise masks for annotated objects (cars, pedestrians, cyclists)
2. **Geometry Guidance (DPT-Hybrid)**: Depth maps for scene structure preservation
3. **Adverse-Weather Synthesis (SDXL + ControlNet-Depth + Inpainting)**: Photorealistic weather effects
4. **Hard-Copy Compositing**: Preserve original objects over synthesized backgrounds
5. **Annotation Preservation**: Copy KITTI labels for synthetic images

### Weather Conditions:
- Heavy rain (streaks + wet road reflections)
- Dense fog/haze (contrast drop)
- Snowfall (flakes + accumulation)
- Night glare (headlight bloom)
- Lens occlusion (water/mud droplets)

### Intensity Levels: mild → moderate → extreme

In [5]:
# ============================================================================
# CONFIGURATION
# ============================================================================
import os
import sys

SEED = 42
DEVICE = 'cuda'

# Preview mode
PREVIEW = False
PREVIEW_SAMPLES = 5

# Model Configuration
BASE_MODEL = "SG161222/RealVisXL_V4.0"
CONTROLNET_DEPTH = "diffusers/controlnet-depth-sdxl-1.0"
SAM_MODEL = "vit_h"  # ViT-H for highest quality

# Weather types and intensity levels
WEATHER_TYPES = ['rain', 'fog', 'snow']
INTENSITY_LEVELS = ['extreme']
K_CANDIDATES = 1

# Quality thresholds
SSIM_THRESHOLD = 0.85
MIN_OBJECTS_PASS = 0.8

# Paths
MOUNT_DRIVE = True
DRIVE_PROJECT_PATH = "/content/drive/MyDrive/Autonomous_Driving_Project"

try:
    from google.colab import drive
    IN_COLAB = True
    if MOUNT_DRIVE:
        drive.mount('/content/drive')
        os.chdir(DRIVE_PROJECT_PATH)
except ImportError:
    IN_COLAB = False
    if os.path.basename(os.getcwd()) == "notebooks":
        os.chdir("..")

PROJECT_ROOT = "."
SYNTHETIC_IMAGES_DIR = "data/synthetic/images"
SYNTHETIC_LABELS_DIR = "data/synthetic/labels"
MANIFEST_PATH = "data/synthetic/manifest.csv"
RESULTS_DIR = "results"

os.makedirs(SYNTHETIC_IMAGES_DIR, exist_ok=True)
os.makedirs(SYNTHETIC_LABELS_DIR, exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/figures", exist_ok=True)

print(f"Mode: {'PREVIEW' if PREVIEW else 'FULL GENERATION'}")
print(f"Weather types: {WEATHER_TYPES}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mode: FULL GENERATION
Weather types: ['rain', 'fog', 'snow']


In [6]:
# Imports
import json
import pickle
import random
import shutil
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm

import torch
from skimage.metrics import structural_similarity as ssim
!pip -q install git+https://github.com/facebookresearch/segment-anything.git
!pip -q install opencv-python pycocotools matplotlib
!mkdir -p checkpoints
!wget -q -O checkpoints/sam_vit_h_4b8939.pth \
  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if DEVICE == 'cuda' and not torch.cuda.is_available():
    DEVICE = 'cpu'
    print("CUDA not available, using CPU")
else:
    print(f"Using device: {DEVICE}")
    if DEVICE == 'cuda':
        print(f"GPU: {torch.cuda.get_device_name(0)}")

  Preparing metadata (setup.py) ... done
Using device: cuda
GPU: NVIDIA A100-SXM4-80GB


## 1. Load Data from Previous Notebooks

In [7]:
scan_report_path = 'results/metrics/scan_report.json'
pairs_path = 'results/metrics/image_label_pairs.pkl'

if not os.path.exists(scan_report_path) or not os.path.exists(pairs_path):
    raise FileNotFoundError("Run 00_setup.ipynb first!")

with open(scan_report_path, 'r') as f:
    scan_report = json.load(f)

with open(pairs_path, 'rb') as f:
    pairs = pickle.load(f)

train_pairs = pairs['train']
print(f"Loaded {len(train_pairs)} training images")

if PREVIEW:
    random.seed(SEED)
    train_pairs = random.sample(train_pairs, min(PREVIEW_SAMPLES, len(train_pairs)))
    print(f"PREVIEW mode: Selected {len(train_pairs)} samples")

Loaded 7481 training images


## 2. Weather Prompt Pools (Extended)

In [8]:
PROMPTS = {
    "rain": {
        "mild": (
            "dashcam view, steady rainfall clearly visible, raindrops streaking across windshield, "
            "wet asphalt with strong reflections, water spray from tires, gray sky, damp scene"
        ),
        "moderate": (
            "dashcam view, moderate-to-heavy rain, thick rain streaks and splashes on windshield, "
            "continuous wiper smears, road shine with mirror-like reflections, strong tire spray, "
            "reduced visibility, hazy distant vehicles"
        ),
        "extreme": (
            "dashcam view, violent torrential downpour, sheets of rain, near-zero visibility, "
            "heavy windshield flooding with large droplets and streaks, aggressive wiper trails, "
            "standing water and deep puddles, hydroplaning spray clouds, dark storm sky"
        ),
    },

    "fog": {
        "mild": (
            "dashcam view, visible morning fog banks, milky haze in the distance, softened contrast, "
            "slightly muted colors, headlight beams faintly visible, background details fading"
        ),
        "moderate": (
            "dashcam view, dense fog, strong atmospheric scattering, low contrast, "
            "headlights create bright cones of light, roadside fades quickly, cars appear as silhouettes, "
            "visibility significantly reduced"
        ),
        "extreme": (
            "dashcam view, extremely thick fog, near-whiteout gray scene, very low visibility, "
            "only close objects visible, headlights bloom and glare heavily, washed-out image, "
            "background fully swallowed by fog"
        ),
    },

    "snow": {
        "mild": (
            "dashcam view, clear snowflakes visible in foreground, light accumulation on roadside, "
            "cold winter lighting, frosty edges, slightly slick road surface"
        ),
        "moderate": (
            "dashcam view, active snowfall, thick snowflakes crossing lens, visibility reduced, "
            "snowy road with tire tracks, slush spray, icy patches, muted colors, overcast winter storm"
        ),
        "extreme": (
            "dashcam view, blizzard, whiteout conditions, violent wind-driven snow, "
            "swirling snow curtains, almost no horizon, heavy accumulation, frozen slush splatter, "
            "extreme low visibility, headlights diffused into bright glow"
        ),
    }
}

NEGATIVE_PROMPT = (
    "clear sunny day, bright blue sky, perfect visibility, dry road, "
    "clean lens, pristine sharp image, "
    "cartoon, anime, illustration, CGI, stylized render, watermark, text, logo, "
    "low-res, jpeg artifacts, over-smoothed, plastic look, "
    "deformed vehicles, melted shapes, duplicated objects, missing wheels, "
    "extra limbs, distorted pedestrians, unrealistic reflections"
)

print("Weather prompts loaded:")
for wt in WEATHER_TYPES:
    print(f"  {wt}: {len(INTENSITY_LEVELS)} intensity levels")

Weather prompts loaded:
  rain: 1 intensity levels
  fog: 1 intensity levels
  snow: 1 intensity levels


## 3. Model Loading: SAM ViT-H + DPT-Hybrid + SDXL

In [9]:
GPU_AVAILABLE = torch.cuda.is_available() and DEVICE == 'cuda'
USE_SAM = False
USE_DPT = False
USE_SDXL = False

sam_predictor = None
dpt_model = None
dpt_processor = None
sdxl_pipeline = None
controlnet = None

print("=" * 60)
print("MODEL LOADING")
print("=" * 60)

if not GPU_AVAILABLE:
    print("[WARN] GPU not available. Using OpenCV fallback.")
else:
    print(f"[GPU] {torch.cuda.get_device_name(0)}")
    print(f"[VRAM] {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    # 1. SAM ViT-H
    try:
        from segment_anything import sam_model_registry, SamPredictor
        import urllib.request

        SAM_CHECKPOINT = "sam_vit_h_4b8939.pth"
        SAM_URL = f"https://dl.fbaipublicfiles.com/segment_anything/{SAM_CHECKPOINT}"

        if not os.path.exists(SAM_CHECKPOINT):
            print(f"   Downloading SAM ViT-H checkpoint...")
            urllib.request.urlretrieve(SAM_URL, SAM_CHECKPOINT)

        sam = sam_model_registry["vit_h"](checkpoint=SAM_CHECKPOINT)
        sam.to(device=DEVICE)
        sam_predictor = SamPredictor(sam)
        USE_SAM = True
        print("[OK] SAM ViT-H loaded")
    except Exception as e:
        print(f"[WARN] SAM unavailable: {e}")

    # 2. DPT-Hybrid Depth
    try:
        from transformers import DPTImageProcessor, DPTForDepthEstimation

        dpt_processor = DPTImageProcessor.from_pretrained("Intel/dpt-hybrid-midas")
        dpt_model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas")
        dpt_model.to(DEVICE)
        dpt_model.eval()
        USE_DPT = True
        print("[OK] DPT-Hybrid Depth loaded")
    except Exception as e:
        print(f"[WARN] DPT unavailable: {e}")

    # 3. SDXL + ControlNet-Depth + Inpainting
    try:
        from diffusers import (
            StableDiffusionXLControlNetInpaintPipeline,
            ControlNetModel,
            AutoencoderKL
        )

        print(f"   Loading ControlNet-Depth...")
        controlnet = ControlNetModel.from_pretrained(
            CONTROLNET_DEPTH,
            torch_dtype=torch.float16,
            variant="fp16"
        )

        print(f"   Loading VAE...")
        vae = AutoencoderKL.from_pretrained(
            "madebyollin/sdxl-vae-fp16-fix",
            torch_dtype=torch.float16
        )

        print(f"   Loading {BASE_MODEL}...")
        sdxl_pipeline = StableDiffusionXLControlNetInpaintPipeline.from_pretrained(
            BASE_MODEL,
            controlnet=controlnet,
            vae=vae,
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True
        )
        sdxl_pipeline.to(DEVICE)

        try:
            sdxl_pipeline.enable_xformers_memory_efficient_attention()
        except:
            sdxl_pipeline.enable_attention_slicing()

        USE_SDXL = True
        print("[OK] SDXL + ControlNet-Depth + Inpainting loaded")
    except Exception as e:
        print(f"[WARN] SDXL unavailable: {e}")

print("\n" + "=" * 60)
print("PIPELINE SUMMARY")
print("=" * 60)
print(f"SAM ViT-H: {'Enabled' if USE_SAM else 'Disabled (bbox fallback)'}")
print(f"DPT Depth: {'Enabled' if USE_DPT else 'Disabled'}")
print(f"SDXL Generation: {'Enabled' if USE_SDXL else 'OpenCV fallback'}")

MODEL LOADING
[GPU] NVIDIA A100-SXM4-80GB
[VRAM] 85.2 GB
[OK] SAM ViT-H loaded


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/414 [00:00<?, ?it/s]

[OK] DPT-Hybrid Depth loaded


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


...

Loading weights:   0%|          | 0/517 [00:00<?, ?it/s]

CLIPTextModelWithProjection LOAD REPORT from: /root/.cache/huggingface/hub/models--SG161222--RealVisXL_V4.0/snapshots/26dfe44930964cd70d0a817b6d1cc945c130e38d/text_encoder_2
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--SG161222--RealVisXL_V4.0/snapshots/26dfe44930964cd70d0a817b6d1cc945c130e38d/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[OK] SDXL + ControlNet-Depth + Inpainting loaded

PIPELINE SUMMARY
SAM ViT-H: Enabled
DPT Depth: Enabled
SDXL Generation: Enabled


## 4. Helper Functions

In [10]:
def parse_kitti_label(label_path):
    """Parse KITTI format label file."""
    objects = []
    try:
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 15:
                    continue
                obj = {
                    'class_name': parts[0],
                    'truncation': float(parts[1]),
                    'occlusion': int(float(parts[2])),
                    'alpha': float(parts[3]),
                    'bbox': (float(parts[4]), float(parts[5]), float(parts[6]), float(parts[7])),
                    'raw_line': line.strip()
                }
                objects.append(obj)
    except Exception as e:
        print(f"Error parsing {label_path}: {e}")
    return objects

def create_bbox_mask(img_shape, objects, dilation=10):
    """Create mask from bounding boxes."""
    h, w = img_shape[:2]
    mask = np.zeros((h, w), dtype=np.float32)

    for obj in objects:
        if obj['class_name'] == 'DontCare':
            continue
        x1, y1, x2, y2 = [int(v) for v in obj['bbox']]
        x1, y1 = max(0, x1 - dilation), max(0, y1 - dilation)
        x2, y2 = min(w, x2 + dilation), min(h, y2 + dilation)
        mask[y1:y2, x1:x2] = 1.0

    return mask

def get_sam_mask(image, objects):
    """Get precise masks using SAM ViT-H."""
    if not USE_SAM:
        return create_bbox_mask(image.shape, objects)

    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.float32)
    sam_predictor.set_image(image)

    for obj in objects:
        if obj['class_name'] == 'DontCare':
            continue
        x1, y1, x2, y2 = [int(v) for v in obj['bbox']]
        input_box = np.array([x1, y1, x2, y2])

        try:
            masks, scores, _ = sam_predictor.predict(
                box=input_box[None, :],
                multimask_output=False
            )
            mask = np.maximum(mask, masks[0].astype(np.float32))
        except:
            mask[y1:y2, x1:x2] = 1.0

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)
    return mask

def get_depth_map(image):
    """Get depth map using DPT-Hybrid."""
    if not USE_DPT:
        return None

    pil_image = Image.fromarray(image)
    inputs = dpt_processor(images=pil_image, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = dpt_model(**inputs)
        depth = outputs.predicted_depth

    depth = torch.nn.functional.interpolate(
        depth.unsqueeze(1),
        size=image.shape[:2],
        mode="bicubic"
    ).squeeze().cpu().numpy()

    depth = (depth - depth.min()) / (depth.max() - depth.min() + 1e-8)
    depth_rgb = np.stack([depth] * 3, axis=-1)
    return (depth_rgb * 255).astype(np.uint8)

print("Helper functions defined")

Helper functions defined


## 5. Weather Generation Functions

In [11]:
# OpenCV fallback functions
def add_rain_opencv(img, intensity=0.5):
    result = img.copy().astype(np.float32)
    h, w = result.shape[:2]
    result *= (0.6 + 0.2 * (1 - intensity))
    n_drops = int(500 * intensity)
    for _ in range(n_drops):
        x, y = np.random.randint(0, w), np.random.randint(0, h)
        length = np.random.randint(10, 30)
        cv2.line(result, (x, y), (x, min(y+length, h-1)), (200, 200, 220), 1)
    return np.clip(result, 0, 255).astype(np.uint8)

def add_fog_opencv(img, intensity=0.5):
    result = img.copy().astype(np.float32)
    h, w = result.shape[:2]
    fog_color = np.array([220, 220, 230], dtype=np.float32)
    fog_density = np.linspace(0.2 * intensity, 0.8 * intensity, h)[:, None, None]
    result = result * (1 - fog_density) + fog_color * fog_density
    return np.clip(result, 0, 255).astype(np.uint8)

def add_snow_opencv(img, intensity=0.5):
    result = img.copy().astype(np.float32)
    h, w = result.shape[:2]
    result = result * 0.9 + 30
    n_flakes = int(1000 * intensity)
    for _ in range(n_flakes):
        x, y = np.random.randint(0, w), np.random.randint(0, h)
        cv2.circle(result, (x, y), np.random.randint(1, 4), (255, 255, 255), -1)
    return np.clip(result, 0, 255).astype(np.uint8)

def add_night_opencv(img, intensity=0.5):
    result = img.copy().astype(np.float32)
    result *= (0.3 + 0.3 * (1 - intensity))
    result[:, :, 2] = np.clip(result[:, :, 2] * 1.2, 0, 255)
    return np.clip(result, 0, 255).astype(np.uint8)

def add_lens_occlusion_opencv(img, intensity=0.5):
    result = img.copy()
    h, w = result.shape[:2]
    overlay = result.copy()
    n_drops = int(20 * intensity)
    for _ in range(n_drops):
        cx, cy = np.random.randint(0, w), np.random.randint(0, h)
        r = np.random.randint(10, 50)
        cv2.circle(overlay, (cx, cy), r, (180, 180, 180), -1)
    result = cv2.addWeighted(result, 0.7, overlay, 0.3, 0)
    return result

OPENCV_WEATHER = {
    'rain': add_rain_opencv,
    'fog': add_fog_opencv,
    'snow': add_snow_opencv,
    'night': add_night_opencv,
}

print("OpenCV weather functions defined")

OpenCV weather functions defined


In [12]:
def generate_weather_sdxl(image, object_mask, weather_type, intensity_level, seed=42):
    """Generate weather using SDXL + ControlNet-Depth + Inpainting."""
    intensity_map = {'mild': 0.3, 'moderate': 0.5, 'extreme': 0.8}
    intensity = intensity_map.get(intensity_level, 0.5)

    if not USE_SDXL:
        weather_func = OPENCV_WEATHER.get(weather_type, add_fog_opencv)
        weather_img = weather_func(image, intensity)
        mask_3ch = object_mask[:, :, np.newaxis]
        return (image * mask_3ch + weather_img * (1 - mask_3ch)).astype(np.uint8)

    prompt = f"RAW photo, realistic photograph, autonomous driving scene, {PROMPTS[weather_type][intensity_level]}, professional photography, 8k uhd"

    pil_image = Image.fromarray(image)
    original_size = pil_image.size

    inpaint_mask = 1.0 - object_mask
    pil_mask = Image.fromarray((inpaint_mask * 255).astype(np.uint8)).convert("L")

    target_size = (1024, 1024)
    pil_image_resized = pil_image.resize(target_size, Image.LANCZOS)
    pil_mask_resized = pil_mask.resize(target_size, Image.NEAREST)

    # Get depth control image
    if USE_DPT:
        depth_map = get_depth_map(np.array(pil_image_resized))
        control_image = Image.fromarray(depth_map)
    else:
        gray = cv2.cvtColor(np.array(pil_image_resized), cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        control_image = Image.fromarray(np.stack([edges]*3, axis=-1))

    generator = torch.Generator(device=DEVICE).manual_seed(seed)
    strength = 0.5 + intensity * 0.30

    try:
        result = sdxl_pipeline(
            prompt=prompt,
            negative_prompt=NEGATIVE_PROMPT,
            image=pil_image_resized,
            mask_image=pil_mask_resized,
            control_image=control_image,
            controlnet_conditioning_scale=0.7,
            strength=strength,
            num_inference_steps=30,
            generator=generator,
            guidance_scale=7.5
        ).images[0]
    except Exception as e:
        print(f"SDXL generation failed: {e}, using OpenCV fallback")
        weather_func = OPENCV_WEATHER.get(weather_type, add_fog_opencv)
        weather_img = weather_func(image, intensity)
        mask_3ch = object_mask[:, :, np.newaxis]
        return (image * mask_3ch + weather_img * (1 - mask_3ch)).astype(np.uint8)

    result = result.resize(original_size, Image.LANCZOS)
    result_np = np.array(result)

    # Hard-copy compositing: preserve original objects
    mask_3ch = object_mask[:, :, np.newaxis]
    final = image * mask_3ch + result_np * (1 - mask_3ch)

    return final.astype(np.uint8)

print("SDXL weather generation function defined")

SDXL weather generation function defined


## 6. Quality Scoring

In [13]:
def compute_preservation_score(original, synthetic, objects):
    """Compute SSIM on object crops."""
    scores = []
    h, w = original.shape[:2]

    for obj in objects:
        if obj['class_name'] == 'DontCare':
            continue
        x1, y1, x2, y2 = [int(v) for v in obj['bbox']]
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)

        if x2 - x1 < 7 or y2 - y1 < 7:
            continue

        orig_crop = cv2.cvtColor(original[y1:y2, x1:x2], cv2.COLOR_RGB2GRAY)
        synth_crop = cv2.cvtColor(synthetic[y1:y2, x1:x2], cv2.COLOR_RGB2GRAY)

        try:
            score = ssim(orig_crop, synth_crop)
            scores.append(score)
        except:
            scores.append(1.0)

    if not scores:
        return 1.0, 1.0

    mean_score = np.mean(scores)
    passed_ratio = sum(1 for s in scores if s >= SSIM_THRESHOLD) / len(scores)
    return mean_score, passed_ratio

print("Quality scoring functions defined")

Quality scoring functions defined


## 7. Best-of-K Generation Pipeline

In [14]:
def process_image(pair, weather_type, intensity_level, k=K_CANDIDATES):
    """Process single image through pipeline."""
    img_path = pair['image']
    label_path = pair['label']
    base_name = pair['base_name']

    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    objects = parse_kitti_label(label_path)
    object_mask = get_sam_mask(img, objects)

    best_candidate = None
    best_score = -1

    for i in range(k):
        seed = SEED + i * 100
        synth_img = generate_weather_sdxl(img, object_mask, weather_type, intensity_level, seed)

        preservation, passed_ratio = compute_preservation_score(img, synth_img, objects)

        if passed_ratio >= MIN_OBJECTS_PASS and preservation > best_score:
            best_score = preservation
            best_candidate = {
                'image': synth_img,
                'seed': seed,
                'preservation_score': preservation,
                'passed_ratio': passed_ratio
            }

    if best_candidate is None:
        synth_img = generate_weather_sdxl(img, object_mask, weather_type, intensity_level, SEED)
        preservation, passed_ratio = compute_preservation_score(img, synth_img, objects)
        best_candidate = {
            'image': synth_img,
            'seed': SEED,
            'preservation_score': preservation,
            'passed_ratio': passed_ratio
        }

    return {
        'original_path': img_path,
        'label_path': label_path,
        'base_name': base_name,
        'weather_type': weather_type,
        'intensity_level': intensity_level,
        'original_img': img,
        'synthetic_img': best_candidate['image'],
        'seed': best_candidate['seed'],
        'preservation_score': best_candidate['preservation_score'],
        'n_objects': len([o for o in objects if o['class_name'] != 'DontCare'])
    }

print("Processing pipeline defined")

Processing pipeline defined


## 8. Run Generation

In [ ]:
print("\n" + "=" * 60)
print(f"SYNTHETIC GENERATION {'(PREVIEW)' if PREVIEW else '(FULL)'}")
print("=" * 60)
print(f"Images: {len(train_pairs)}")
print(f"Weather types: {WEATHER_TYPES}")
print(f"Intensity levels: {INTENSITY_LEVELS}")

results = []
manifest_data = []

for pair in tqdm(train_pairs, desc="Generating"):
    weather_type = random.choice(WEATHER_TYPES)
    intensity_level = random.choice(INTENSITY_LEVELS)

    result = process_image(pair, weather_type, intensity_level)
    if result is None:
        continue

    results.append(result)

    synth_filename = f"{result['base_name']}_{weather_type}_{intensity_level}_synth.png"
    manifest_data.append({
        'original_path': result['original_path'],
        'synthetic_path': os.path.join(SYNTHETIC_IMAGES_DIR, synth_filename),
        'weather_type': weather_type,
        'intensity_level': intensity_level,
        'seed': result['seed'],
        'preservation_score': result['preservation_score']
    })
print(f"\nGenerated {len(results)} synthetic images")




SYNTHETIC GENERATION (FULL)
Images: 7481
Weather types: ['rain', 'fog', 'snow']
Intensity levels: ['extreme']


Generating:  22%|██▏       | 1681/7481 [00:00<00:01, 5678.81it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (80 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (80 > 77). Running this sequence through the model will result in indexing errors


  0%|          | 0/22 [00:00<?, ?it/s]

Generating:  22%|██▏       | 1681/7481 [00:12<00:01, 5678.81it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

...

  0%|          | 0/22 [00:00<?, ?it/s]

Generating:  32%|███▏      | 2372/7481 [1:11:57<18:20:24, 12.92s/it]

  0%|          | 0/22 [00:00<?, ?it/s]

Generating:  32%|███▏      | 2373/7481 [1:12:08<17:29:28, 12.33s/it]

  0%|          | 0/22 [00:00<?, ?it/s]

## 9. Visualization (PREVIEW)

In [ ]:
if PREVIEW and len(results) > 0:
    n_display = min(len(results), PREVIEW_SAMPLES)
    fig, axes = plt.subplots(n_display, 2, figsize=(12, 4*n_display))
    if n_display == 1:
        axes = axes.reshape(1, -1)

    for i, result in enumerate(results[:n_display]):
        axes[i, 0].imshow(result['original_img'])
        axes[i, 0].set_title(f"Original: {result['base_name']}")
        axes[i, 0].axis('off')

        axes[i, 1].imshow(result['synthetic_img'])
        axes[i, 1].set_title(
            f"{result['weather_type']} ({result['intensity_level']})\n"
            f"Preservation: {result['preservation_score']:.2f}"
        )
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/figures/synthetic_preview.png", dpi=150)
    plt.show()
    print(f"Preview saved to {RESULTS_DIR}/figures/synthetic_preview.png")

## 10. Save Results (FULL mode)

In [ ]:
if not PREVIEW:
    print("\n" + "=" * 60)
    print("SAVING SYNTHETIC DATA")
    print("=" * 60)

    for result in tqdm(results, desc="Saving"):
        base_name = result['base_name']
        weather_type = result['weather_type']
        intensity = result['intensity_level']

        synth_img_name = f"{base_name}_{weather_type}_{intensity}_synth.png"
        synth_label_name = f"{base_name}_{weather_type}_{intensity}_synth.txt"

        synth_img_path = os.path.join(SYNTHETIC_IMAGES_DIR, synth_img_name)
        synth_label_path = os.path.join(SYNTHETIC_LABELS_DIR, synth_label_name)

        synth_bgr = cv2.cvtColor(result['synthetic_img'], cv2.COLOR_RGB2BGR)
        cv2.imwrite(synth_img_path, synth_bgr)
        shutil.copy2(result['label_path'], synth_label_path)

    manifest_df = pd.DataFrame(manifest_data)
    manifest_df.to_csv(MANIFEST_PATH, index=False)

    print(f"\nSaved {len(results)} synthetic images")
    print(f"Manifest: {MANIFEST_PATH}")

    print("\nStatistics:")
    for wt in WEATHER_TYPES:
        count = len(manifest_df[manifest_df['weather_type'] == wt])
        print(f"  {wt}: {count}")
else:
    print("\nPREVIEW mode: No files saved. Set PREVIEW = False to save.")

## Summary

### Pipeline Steps Completed:
1. **Object Segmentation (SAM ViT-H)**: Precise masks for annotated objects
2. **Geometry Guidance (DPT-Hybrid)**: Depth maps for structure preservation
3. **Adverse-Weather Synthesis (SDXL + ControlNet-Depth)**: Photorealistic weather
4. **Hard-Copy Compositing**: Original objects preserved exactly
5. **Annotation Preservation**: KITTI labels copied for synthetic images

### Weather Conditions:
- Rain (streaks + wet road reflections)
- Fog (contrast drop, visibility reduction)
- Snow (flakes + accumulation)
- Night (low light + headlight glare)
- Lens occlusion (water/mud droplets)

### Next Steps:
- Run `03_train_eval_compare.ipynb` for YOLOv8s training and robustness evaluation